In [ ]:
import os
from typing import Type

import contextily as cx
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# from sklearn.metrics import mean_squared_error as mse
# from splot.libpysal import plot_spatial_weights
import prince
import scipy.stats as stats

# from splot.esda import moran_scatterplot
import seaborn as sns
import statsmodels
import statsmodels.api as sm
from pysal.lib import weights

# from pysal.explore import esda
# from pysal.viz import splot
from pysal.model import spreg

# from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from splot.esda import moran_scatterplot
from statsmodels.graphics.gofplots import ProbPlot, qqplot
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import maybe_unwrap_results
from ydata_profiling import ProfileReport

%matplotlib inline
os.environ["USE_PYGEOS"] = "0"

# Modelling (AIC)


## 1 Data Preparation for Modelling


### 1.1 Loading and Transforming Datasets


In [ ]:
all_data = pd.read_pickle("../Data/all_data.piclo")

In [ ]:
all_data.shape

In [ ]:
# # filtro de outliers, calculados ao longo das regressões realizadas no notebook 3 (Modelling) e adicionado ao longo do processo
# # (linha acima foram usadas para atualizar esta lista até o retorno de um conjunto vazio)
# # a cada "run" do notebook, esta lista deve ser atualizada com os outliers identificados
# # parou-se de atualizar a lista quando o conjunto de outliers começou, repetidamente, a devolver apenas 2 ou 3 outliers

# run 1 - 13 outliers
all_data = all_data.loc[all_data["ID"] != 563242]
all_data = all_data.loc[all_data["ID"] != 1177680]
all_data = all_data.loc[all_data["ID"] != 1131818]
all_data = all_data.loc[all_data["ID"] != 2383113]
all_data = all_data.loc[all_data["ID"] != 2568707]
all_data = all_data.loc[all_data["ID"] != 2639959]
all_data = all_data.loc[all_data["ID"] != 2632030]
all_data = all_data.loc[all_data["ID"] != 2870116]
all_data = all_data.loc[all_data["ID"] != 2743956]
all_data = all_data.loc[all_data["ID"] != 2939163]
all_data = all_data.loc[all_data["ID"] != 2953594]
all_data = all_data.loc[all_data["ID"] != 2939578]
all_data = all_data.loc[all_data["ID"] != 3089766]

# run 2 - 12 outliers
all_data = all_data.loc[all_data["ID"] != 563276]
all_data = all_data.loc[all_data["ID"] != 1254649]
all_data = all_data.loc[all_data["ID"] != 1253964]
all_data = all_data.loc[all_data["ID"] != 2251268]
all_data = all_data.loc[all_data["ID"] != 2569071]
all_data = all_data.loc[all_data["ID"] != 2644045]
all_data = all_data.loc[all_data["ID"] != 2631437]
all_data = all_data.loc[all_data["ID"] != 2712461]
all_data = all_data.loc[all_data["ID"] != 2696442]
all_data = all_data.loc[all_data["ID"] != 2939264]
all_data = all_data.loc[all_data["ID"] != 2811941]
all_data = all_data.loc[all_data["ID"] != 3161153]

# run 3 - 12 outliers
all_data = all_data.loc[all_data["ID"] != 563391]
all_data = all_data.loc[all_data["ID"] != 1235755]
all_data = all_data.loc[all_data["ID"] != 1247318]
all_data = all_data.loc[all_data["ID"] != 2380527]
all_data = all_data.loc[all_data["ID"] != 2569178]
all_data = all_data.loc[all_data["ID"] != 2638746]
all_data = all_data.loc[all_data["ID"] != 2629208]
all_data = all_data.loc[all_data["ID"] != 2718713]
all_data = all_data.loc[all_data["ID"] != 2875618]
all_data = all_data.loc[all_data["ID"] != 2687064]
all_data = all_data.loc[all_data["ID"] != 2958943]
all_data = all_data.loc[all_data["ID"] != 3120730]

In [ ]:
all_data

In [ ]:
all_data.dtypes

In [ ]:
all_data["Cluster_LP"].unique()

In [ ]:
all_data[all_data.isnull().any(axis=1)]

In [ ]:
# convertion of data
all_data["Year"] = all_data["Year"].astype("int64")
all_data["Cluster_LP"] = all_data["Cluster_LP"].astype("int64")

In [ ]:
all_data.shape

In [ ]:
all_data["T"].sum()

In [ ]:
# a=set(outliers_sum)
# all_data.loc[list(a)]

In [ ]:
all_data.shape

In [ ]:
all_data["T"].value_counts()

In [ ]:
all_data["Year"].value_counts()

In [ ]:
# valores do Indice de Preços da Habitação para Aveiro (valores para 21, 22 e 23 são previsões, com base na tendência dos últimos anos)
IPI2005 = 116.5
IPI2006 = 110.9
IPI2007 = 114.3
IPI2008 = 109.4
IPI2009 = 103.1
IPI2010 = 104.4
IPI2018 = 100.8
IPI2019 = 114.2
IPI2020 = 126.2
IPI2021 = 132.1
IPI2022 = 133.7
IPI2023 = 135.4

In [ ]:
# write IPI values to new column
def new_column_value(Year):
    if Year == 2005:
        return IPI2005
    elif Year == 2006:
        return IPI2006
    elif Year == 2007:
        return IPI2007
    elif Year == 2008:
        return IPI2008
    elif Year == 2009:
        return IPI2009
    elif Year == 2010:
        return IPI2010
    elif Year == 2018:
        return IPI2018
    elif Year == 2019:
        return IPI2019
    elif Year == 2020:
        return IPI2020
    elif Year == 2021:
        return IPI2021
    elif Year == 2022:
        return IPI2022
    elif Year == 2023:
        return IPI2023


all_data["IPI"] = all_data["Year"].apply(new_column_value)

In [ ]:
all_data.shape

In [ ]:
# valores da Taxa Anual de juro (TAA) de novos empréstimos à habitação (BdP)
TAA2005 = 3.38
TAA2006 = 4.01
TAA2007 = 4.8
TAA2008 = 5.44
TAA2009 = 2.73
TAA2010 = 2.47
TAA2018 = 1.41
TAA2019 = 1.22
TAA2020 = 1
TAA2021 = 0.81
TAA2022 = 1.82
TAA2023 = 3.76

In [ ]:
# write IPI values to new column
def new_column_value(Year):
    if Year == 2005:
        return TAA2005
    elif Year == 2006:
        return TAA2006
    elif Year == 2007:
        return TAA2007
    elif Year == 2008:
        return TAA2008
    elif Year == 2009:
        return TAA2009
    elif Year == 2010:
        return TAA2010
    elif Year == 2018:
        return TAA2018
    elif Year == 2019:
        return TAA2019
    elif Year == 2020.0:
        return TAA2020
    elif Year == 2021.0:
        return TAA2021
    elif Year == 2022:
        return TAA2022
    elif Year == 2023:
        return TAA2023


all_data["TAA"] = all_data["Year"].apply(new_column_value)

In [ ]:
all_data.shape

In [ ]:
## poucos valores para os anos 2023 e 2005 (início de uma base de dados e fim da outra - serão removidos)

all_data = all_data.loc[all_data["Year"] != 2023]
all_data = all_data.loc[all_data["Year"] != 2005]

In [ ]:
all_data.shape

In [ ]:
all_data.columns

In [ ]:
a = list(all_data[["Tot_AL", "IPI", "TAA"]])

In [ ]:
# Standardizing the features
scaler = StandardScaler()
all_data[a] = StandardScaler().fit_transform(all_data[a])

In [ ]:
all_data.head()

### 1.2 MCA for Intrinsic Features

https://maxhalford.github.io/prince/mca/ - Multiple correspondence analysis


In [ ]:
all_data.head()

In [ ]:
all_data.columns

In [ ]:
a = ["Typology", "Nature", "Status"]

In [ ]:
a
# OK

In [ ]:
mca = prince.MCA(
    n_components=10,
    n_iter=3,
    copy=True,
    check_input=True,
    engine="sklearn",
    random_state=42,
)
mca.fit(all_data[a])

In [ ]:
mca.eigenvalues_summary

In [ ]:
mca.column_contributions_.style.format("{:.0%}")

In [ ]:
PC_values = np.arange(mca.n_components) + 1
plt.plot(
    PC_values,
    [17.15, 11.88, 10.52, 10.11, 10, 10, 9.88, 9.30, 8.06, 3.10],
    "o-",
    linewidth=2,
    color="blue",
)
plt.axhline(10.14, color="green", linestyle="--", linewidth=1)
plt.title("Scree Plot")
plt.xlabel("Componentes Principais")
plt.ylabel("Variancia Explicada")
plt.show()

In [ ]:
all_data["MCA_1"] = mca.transform(all_data[a])[0]
all_data["MCA_2"] = mca.transform(all_data[a])[1]
all_data["MCA_3"] = mca.transform(all_data[a])[2]
all_data["MCA_4"] = mca.transform(all_data[a])[3]

In [ ]:
all_data.tail()

In [ ]:
all_data.shape

In [ ]:
all_data.reset_index(inplace=True, drop=True)

### 1.3 Generate Discriptive Statistics HTML


In [ ]:
all_data.columns

In [ ]:
all_data["Nature"] = all_data["Nature"].astype("category")
all_data["Typology"] = all_data["Typology"].astype("category")
all_data["Status"] = all_data["Status"].astype("category")

In [ ]:
# Generate the report - Final

all_data_analysis = all_data.copy()
all_data_analysis = all_data_analysis[
    ["Log_P_A", "MCA_1", "MCA_2", "MCA_3", "MCA_4", "TAA", "IPI", "Tot_AL"]
]
profile = ProfileReport(
    all_data_analysis,
    title="AIC Data Profile Report",
    explorative=True,
    config_file="../Data/config_default.yaml",
)
profile.to_notebook_iframe()

In [ ]:
# # Save the report to .html
profile.to_file("AIC_Data_Profile_Report_MCA.html")

In [ ]:
# Get min, max, mean, std dev for the dataset

all_data.describe()

In [ ]:
all_data.describe()

### 1.4 Create GeoDataFrames with all_data + different territorial limits (Freguesias, Skater Cluster, Cluster LP)


#### 1.4.1 Create GeoDataFrames with all_data + Cluster_LP


In [ ]:
Cluster_LP = pd.read_pickle("../Data/piclo_clusters_2.piclo")

In [ ]:
Cluster_LP.head()

In [ ]:
Cluster_LP.shape

In [ ]:
all_data.shape

In [ ]:
all_data = all_data[~all_data["Cluster_LP"].isnull()]

In [ ]:
all_data.shape

In [ ]:
# merge dwelling data with clusters data
all_data_LP = all_data.merge(Cluster_LP, on="Cluster_LP", how="left")

In [ ]:
all_data_LP.columns

In [ ]:
all_data_LP.shape

In [ ]:
# drop unnecessary columns
all_data_LP.drop(
    columns=[
        "PCA_1",
        "PCA_2",
        "PCA_3",
        "PCA_4",
        "PCA_5",
        "PCA_6",
        "PCA_7",
        "PCA_8",
        "PCA_9",
        "PCA_10",
        "PCA_11",
        "PCA_12",
        "PCA_13",
        "PCA_14",
        "PCA_15",
        "PCA_16",
        "PCA_17",
        "tot_cs",
        "tot_py",
        "tot_min",
        "Price",
    ],
    axis=1,
    inplace=True,
)

In [ ]:
# drop dwellings with no lat lon information (geometry)
all_data_LP2 = all_data_LP[all_data_LP["geometry"].isna()]

In [ ]:
all_data_LP2["geometry"].unique()

In [ ]:
all_data_LP2.columns

In [ ]:
all_data_LP2["Cluster_LP"].unique()

In [ ]:
# drop dwellings with no lat lon information (geometry)
all_data_LP = all_data_LP[~all_data_LP["geometry"].isna()]

In [ ]:
all_data_LP.shape

In [ ]:
a = all_data_LP[all_data_LP["T"] == 1]
a

In [ ]:
all_data_LP.columns

In [ ]:
len(all_data_LP["Cluster_LP"].unique())

In [ ]:
all_data_LP.dtypes

In [ ]:
# convertion of data to float
all_data_LP["Zona_Ward"] = np.floor(
    pd.to_numeric(all_data_LP["Zona_Ward"], errors="coerce")
).astype("float64")
all_data_LP["Zona_Ward_Queen"] = np.floor(
    pd.to_numeric(all_data_LP["Zona_Ward_Queen"], errors="coerce")
).astype("float64")
all_data_LP["Zona_Maxp"] = np.floor(
    pd.to_numeric(all_data_LP["Zona_Maxp"], errors="coerce")
).astype("float64")
all_data_LP["Zona_SKATER"] = np.floor(
    pd.to_numeric(all_data_LP["Zona_SKATER"], errors="coerce")
).astype("float64")
all_data_LP["Cluster_LP"] = np.floor(
    pd.to_numeric(all_data_LP["Cluster_LP"], errors="coerce")
).astype("float64")
all_data_LP["Year"] = np.floor(
    pd.to_numeric(all_data_LP["Year"], errors="coerce")
).astype("float64")
all_data_LP["T"] = np.floor(pd.to_numeric(all_data_LP["T"], errors="coerce")).astype(
    "float64"
)

In [ ]:
all_data_LP.dtypes

In [ ]:
all_data_LP.head()

In [ ]:
all_data_LP.columns

In [ ]:
all_data_LP.shape

In [ ]:
all_data_LP["geometry"].nunique()

#### 1.4.2 Create GeoDataFrames with all_data + Skater


In [ ]:
all_data_skater = all_data_LP.copy()

In [ ]:
all_data_skater.isnull().values.any()

In [ ]:
all_data_skater.columns

In [ ]:
all_data_skater = gpd.GeoDataFrame(all_data_skater, geometry="geometry")

In [ ]:
all_data_skater_geo = all_data_skater.dissolve(by="Zona_SKATER").reset_index()

In [ ]:
all_data_skater_geo = all_data_skater_geo[["Zona_SKATER", "geometry"]]

In [ ]:
all_data_skater = all_data_skater.merge(
    all_data_skater_geo, on="Zona_SKATER", how="left"
)

In [ ]:
all_data_skater.shape

In [ ]:
all_data_skater.columns

In [ ]:
all_data_skater.drop(columns=["geometry_x"], axis=1, inplace=True)
all_data_skater.rename(columns={"geometry_y": "geometry"}, inplace=True)

In [ ]:
all_data_skater = gpd.GeoDataFrame(all_data_skater, geometry="geometry")

In [ ]:
all_data_skater.shape

In [ ]:
all_data_skater = all_data_skater[~all_data_skater["Cluster_LP"].isnull()]
all_data_skater = all_data_skater[~all_data_skater["Zona_SKATER"].isnull()]

In [ ]:
all_data_skater.shape

In [ ]:
len(all_data_skater["Zona_SKATER"].unique())

In [ ]:
all_data_skater.dtypes

In [ ]:
all_data_skater.head()

In [ ]:
all_data_skater.columns

In [ ]:
all_data_skater.shape

In [ ]:
all_data_skater["geometry"].nunique()

In [ ]:
all_data_skater["Year"].nunique()

#### 1.4.3 Create GeoDataFrames with all_data + FR


In [ ]:
all_data_fr = all_data_LP.copy()

In [ ]:
# transform data to geodataframe
all_data_fr = gpd.GeoDataFrame(all_data_fr, geometry="geometry")

In [ ]:
all_data_fr.plot(
    column="Cluster_LP", categorical=True, legend=False, figsize=(10, 10), cmap="tab20c"
)

In [ ]:
all_data_fr.columns

In [ ]:
CLUSTER_FR = pd.read_pickle("../Data/piclo_clusters_fr.piclo")

In [ ]:
CLUSTER_FR

In [ ]:
CLUSTER_FR.plot()

In [ ]:
CLUSTER_LP = pd.read_pickle("../Data/piclo_clusters_lp.piclo")

In [ ]:
CLUSTER_LP.plot()

In [ ]:
CLUSTER_LP["geometry2"] = CLUSTER_LP.centroid

In [ ]:
CLUSTER_LP.rename(
    columns={"geometry": "geometry2", "geometry2": "geometry"}, inplace=True
)
CLUSTER_LP.drop(columns=["geometry2"], inplace=True)

In [ ]:
CLUSTER_LP

In [ ]:
CLUSTER = CLUSTER_FR.sjoin(CLUSTER_LP, how="inner", predicate="intersects")

In [ ]:
CLUSTER.shape

In [ ]:
CLUSTER.head()

In [ ]:
CLUSTER["Cluster_LP"] = CLUSTER["Cluster_LP"].astype("float64")
CLUSTER["FR11"] = CLUSTER["FR11"].astype("Int64")

In [ ]:
CLUSTER = CLUSTER[["Cluster_LP", "FR11", "geometry"]]

In [ ]:
all_data_fr.shape

In [ ]:
all_data_fr.head()

In [ ]:
all_data_fr = all_data_fr.merge(CLUSTER, on="Cluster_LP", how="left")

In [ ]:
all_data_fr.columns

In [ ]:
all_data_fr.drop(columns=["geometry_x"], axis=1, inplace=True)
all_data_fr.rename(columns={"geometry_y": "geometry"}, inplace=True)
all_data_fr = all_data_fr[~all_data_fr["Cluster_LP"].isnull()]
all_data_fr = all_data_fr[~all_data_fr["Zona_SKATER"].isnull()]
all_data_fr = all_data_fr[~all_data_fr["FR11"].isnull()]

In [ ]:
all_data_fr.head()

In [ ]:
all_data_fr.shape

In [ ]:
all_data_fr["geometry"].nunique()

## 2 Linear Regressions (Aveiro) - SKATER


### 2.1 Data Preparation for DID Linear Regression


In [ ]:
# transform data to geodataframe
data_aveiro_skater = gpd.GeoDataFrame(all_data_skater, geometry="geometry")

In [ ]:
data_aveiro_skater.columns

In [ ]:
data_aveiro_skater["T"].value_counts()

In [ ]:
# view Zona_SKATER clusters
ax = data_aveiro_skater.plot(
    figsize=(10, 10),
    column="Zona_SKATER",
    categorical=True,
    edgecolor="w",
    legend=True,
    linewidth=0.2,
    cmap="tab20",
)
cx.add_basemap(ax, crs=data_aveiro_skater.crs, source=cx.providers.OpenStreetMap.Mapnik)

In [ ]:
# apply above list to data
data_aveiro_skater["D"] = np.where(
    (data_aveiro_skater["Zona_SKATER"] == 3.0)
    | (data_aveiro_skater["Zona_SKATER"] == 6.0)
    | (data_aveiro_skater["Zona_SKATER"] == 7.0),
    1,
    0,
)

In [ ]:
data_aveiro_skater.head()

In [ ]:
data_aveiro_skater.dtypes

In [ ]:
# check no. of dwellings per cluster in Aveiro Center
pd.pivot_table(
    data_aveiro_skater,
    values="Log_P_A",
    index=["Zona_SKATER"],
    aggfunc=lambda x: len(x.unique()),
).head(10)
# não incluir cluster 80, 102 e 106, por escassez de dados

In [ ]:
data_aveiro_skater.to_pickle("../Data/data_aveiro_skater.piclo")

In [ ]:
data_aveiro_skater["D"].value_counts()

In [ ]:
a = data_aveiro_skater[data_aveiro_skater["D"] == 0]
a[["Log_P_A", "Tot_AL", "TAA", "IPI", "MCA_1", "MCA_2", "MCA_3", "MCA_4"]].describe()

In [ ]:
sum(data_aveiro_skater["D"].value_counts())

In [ ]:
# result of the SKATER Regionalization
ax = data_aveiro_skater.plot(
    figsize=(10, 10),
    column=data_aveiro_skater["D"],
    categorical=True,
    legend=True,
    linewidth=0.1,
    cmap="tab20",
)
cx.add_basemap(ax, crs=data_aveiro_skater.crs, source=cx.providers.OpenStreetMap.Mapnik)
ax.set_title(
    "Treatment Group (1) and Control Group (0)", fontweight="bold", fontsize=16
)
ax.set_axis_off()

In [ ]:
# check areas defined as control and intervention areas (D=0 and D=1)
ax = data_aveiro_skater.plot(
    column=data_aveiro_skater["D"],
    categorical=True,
    legend=True,
    figsize=(10, 10),
    cmap="tab20",
)
plt.title("Zona de Intervenção (1) e Zona de Controlo (0)")
cx.add_basemap(ax, crs=data_aveiro_skater.crs, source=cx.providers.OpenStreetMap.Mapnik)

In [ ]:
# calculate DT (true when both D and T are equal to 1)
data_aveiro_skater["DT"] = data_aveiro_skater["D"] * data_aveiro_skater["T"]

In [ ]:
data_aveiro_skater["DT"].value_counts()

In [ ]:
# check log_P_A distribution in the territory
ax = data_aveiro_skater.plot(
    column=np.exp(data_aveiro_skater["Log_P_A"]),
    legend=True,
    figsize=(10, 10),
    cmap="viridis",
    scheme="quantiles",
    k=6,
    linewidth=0.3,
    edgecolor="black",
)
plt.title("Preço médio dos imóveis (€/m2), por Cluster (SKATER)")
cx.add_basemap(ax, crs=data_aveiro_skater.crs, source=cx.providers.OpenStreetMap.Mapnik)

### 2.2 Linear Regression (focused in Aveiro Center - SKATER Cluster)


In [ ]:
data_aveiro_skater.shape

In [ ]:
# check number of dwellings per cluster
pd.pivot_table(
    data_aveiro_skater,
    values="Log_P_A",
    index=["Zona_SKATER"],
    aggfunc=lambda x: len(x.unique()),
).head(10)

In [ ]:
# check number of dwellings per cluster
pd.pivot_table(
    data_aveiro_skater, values="DT", index=["Zona_SKATER"], aggfunc="sum"
).head(10)

In [ ]:
# check number of dwellings per cluster
pd.pivot_table(
    data_aveiro_skater,
    values="Zona_SKATER",
    index=["D"],
    aggfunc=lambda x: len(x.unique()),
)

In [ ]:
data_aveiro_skater.columns

In [ ]:
# get dummies for skater zones
data_aveiro_skater_ols = pd.get_dummies(
    data_aveiro_skater, columns=["Zona_SKATER"], drop_first=True, dtype=float
)

In [ ]:
data_aveiro_skater_ols.columns

In [ ]:
# define X and y
c_y_eur_area_skater = data_aveiro_skater_ols["Log_P_A"].astype(float)

c_X_eur_area_skater = data_aveiro_skater_ols[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "IPI",
        "Tot_AL",
        "TAA",
        "Zona_SKATER_1.0",
        "Zona_SKATER_2.0",
        "Zona_SKATER_3.0",
        "Zona_SKATER_4.0",
        "Zona_SKATER_5.0",
        "Zona_SKATER_6.0",
        "Zona_SKATER_7.0",
        "Zona_SKATER_8.0",
    ]
].astype(float)

In [ ]:
# linear model for casasapo with log of price per square meter as dependent variable
c_X_eur_area_skater = sm.add_constant(c_X_eur_area_skater)
model_c_eur_area_skater = sm.OLS(c_y_eur_area_skater, c_X_eur_area_skater)
results_c_eur_area_skater = model_c_eur_area_skater.fit()
skater_save = results_c_eur_area_skater.summary2()
results_c_eur_area_skater.summary2()

In [ ]:
# # export results to csv
# skater_save.tables[0].to_csv('skater_0.csv')
# skater_save.tables[1].to_csv('skater_1.csv')


In [ ]:
# base code for diagnostic plots

style_talk = "seaborn-talk"  # refer to plt.style.available


class Linear_Reg_Diagnostic:
    """
    Diagnostic plots to identify potential problems in a linear regression fit.
    Mainly,
        a. non-linearity of data
        b. Correlation of error terms
        c. non-constant variance
        d. outliers
        e. high-leverage points
        f. collinearity

    Author:
        Prajwal Kafle (p33ajkafle@gmail.com, where 3 = r)
        Does not come with any sort of warranty.
        Please test the code one your end before using.
    """

    def __init__(
        self,
        results: Type[statsmodels.regression.linear_model.RegressionResultsWrapper],
    ) -> None:
        """
        For a linear regression model, generates following diagnostic plots:

        a. residual
        b. qq
        c. scale location and
        d. leverage

        and a table

        e. vif

        Args:
            results (Type[statsmodels.regression.linear_model.RegressionResultsWrapper]):
                must be instance of statsmodels.regression.linear_model object

        Raises:
            TypeError: if instance does not belong to above object

        Example:
        >>> import numpy as np
        >>> import pandas as pd
        >>> import statsmodels.formula.api as smf
        >>> x = np.linspace(-np.pi, np.pi, 100)
        >>> y = 3*x + 8 + np.random.normal(0,1, 100)
        >>> df = pd.DataFrame({'x':x, 'y':y})
        >>> res = smf.ols(formula= "y ~ x", data=df).fit()
        >>> cls = Linear_Reg_Diagnostic(res)
        >>> cls(plot_context="seaborn-paper")

        In case you do not need all plots you can also independently make an individual plot/table
        in following ways

        >>> cls = Linear_Reg_Diagnostic(res)
        >>> cls.residual_plot()
        >>> cls.qq_plot()
        >>> cls.scale_location_plot()
        >>> cls.leverage_plot()
        >>> cls.vif_table()
        """

        if (
            isinstance(
                results, statsmodels.regression.linear_model.RegressionResultsWrapper
            )
            is False
        ):
            raise TypeError(
                "result must be instance of statsmodels.regression.linear_model.RegressionResultsWrapper object"
            )

        self.results = maybe_unwrap_results(results)

        self.y_true = self.results.model.endog
        self.y_predict = self.results.fittedvalues
        self.xvar = self.results.model.exog
        self.xvar_names = self.results.model.exog_names

        self.residual = np.array(self.results.resid)
        influence = self.results.get_influence()
        self.residual_norm = influence.resid_studentized_internal
        self.leverage = influence.hat_matrix_diag
        self.cooks_distance = influence.cooks_distance[0]
        self.nparams = len(self.results.params)

    def __call__(self, plot_context="seaborn-v0_8-paper"):
        # print(plt.style.available)
        with plt.style.context(plot_context):
            fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
            self.residual_plot(ax=ax[0, 0])
            self.qq_plot(ax=ax[0, 1])
            self.scale_location_plot(ax=ax[1, 0])
            self.leverage_plot(ax=ax[1, 1])
            plt.show()

        self.vif_table()
        return fig, ax

    def residual_plot(self, ax=None):
        """
        Residual vs Fitted Plot

        Graphical tool to identify non-linearity.
        (Roughly) Horizontal red line is an indicator that the residual has a linear pattern
        """
        if ax is None:
            fig, ax = plt.subplots()

        sns.residplot(
            x=self.y_predict,
            y=self.residual,
            lowess=True,
            scatter_kws={"alpha": 0.5},
            line_kws={"color": "red", "lw": 1, "alpha": 0.8},
            ax=ax,
        )

        # annotations
        residual_abs = np.abs(self.residual)
        abs_resid = np.flip(np.sort(residual_abs))
        abs_resid_top_3 = abs_resid[:3]
        for i, _ in enumerate(abs_resid_top_3):
            ax.annotate(i, xy=(self.y_predict[i], self.residual[i]), color="C3")

        ax.set_title("Residuals vs Fitted", fontweight="bold")
        ax.set_xlabel("Fitted values")
        ax.set_ylabel("Residuals")
        return ax

    def qq_plot(self, ax=None):
        """
        Standarized Residual vs Theoretical Quantile plot

        Used to visually check if residuals are normally distributed.
        Points spread along the diagonal line will suggest so.
        """
        if ax is None:
            fig, ax = plt.subplots()

        QQ = ProbPlot(self.residual_norm)
        QQ.qqplot(line="45", alpha=0.5, lw=1, ax=ax)

        # annotations
        abs_norm_resid = np.flip(np.argsort(np.abs(self.residual_norm)), 0)
        abs_norm_resid_top_3 = abs_norm_resid[:3]
        for r, i in enumerate(abs_norm_resid_top_3):
            ax.annotate(
                i,
                xy=(np.flip(QQ.theoretical_quantiles, 0)[r], self.residual_norm[i]),
                ha="right",
                color="C3",
            )

        ax.set_title("Normal Q-Q", fontweight="bold")
        ax.set_xlabel("Theoretical Quantiles")
        ax.set_ylabel("Standardized Residuals")
        return ax

    def scale_location_plot(self, ax=None):
        """
        Sqrt(Standarized Residual) vs Fitted values plot

        Used to check homoscedasticity of the residuals.
        Horizontal line will suggest so.
        """
        if ax is None:
            fig, ax = plt.subplots()

        residual_norm_abs_sqrt = np.sqrt(np.abs(self.residual_norm))

        ax.scatter(self.y_predict, residual_norm_abs_sqrt, alpha=0.5)
        sns.regplot(
            x=self.y_predict,
            y=residual_norm_abs_sqrt,
            scatter=False,
            ci=False,
            lowess=True,
            line_kws={"color": "red", "lw": 1, "alpha": 0.8},
            ax=ax,
        )

        # annotations
        abs_sq_norm_resid = np.flip(np.argsort(residual_norm_abs_sqrt), 0)
        abs_sq_norm_resid_top_3 = abs_sq_norm_resid[:3]
        for i in abs_sq_norm_resid_top_3:
            ax.annotate(
                i, xy=(self.y_predict[i], residual_norm_abs_sqrt[i]), color="C3"
            )
        ax.set_title("Scale-Location", fontweight="bold")
        ax.set_xlabel("Fitted values")
        ax.set_ylabel(r"$\sqrt{|\mathrm{Standardized\ Residuals}|}$")
        return ax

    def leverage_plot(self, ax=None):
        """
        Residual vs Leverage plot

        Points falling outside Cook's distance curves are considered observation that can sway the fit
        aka are influential.
        Good to have none outside the curves.
        """
        if ax is None:
            fig, ax = plt.subplots()

        ax.scatter(self.leverage, self.residual_norm, alpha=0.5)
        sns.regplot(
            x=self.leverage,
            y=self.residual_norm,
            scatter=False,
            ci=False,
            lowess=True,
            line_kws={"color": "red", "lw": 1, "alpha": 0.8},
            ax=ax,
        )

        # annotations
        leverage_top_3 = np.flip(np.argsort(self.cooks_distance), 0)[:3]
        for i in leverage_top_3:
            ax.annotate(i, xy=(self.leverage[i], self.residual_norm[i]), color="C3")

        xtemp, ytemp = self.__cooks_dist_line(0.5)  # 0.5 line
        ax.plot(xtemp, ytemp, label="Cook's distance", lw=1, ls="--", color="red")
        xtemp, ytemp = self.__cooks_dist_line(1)  # 1 line
        ax.plot(xtemp, ytemp, lw=1, ls="--", color="red")

        ax.set_xlim(0, max(self.leverage) + 0.01)
        ax.set_title("Residuals vs Leverage", fontweight="bold")
        ax.set_xlabel("Leverage")
        ax.set_ylabel("Standardized Residuals")
        ax.legend(loc="upper right")
        return ax

    def vif_table(self):
        """
        VIF table

        VIF, the variance inflation factor, is a measure of multicollinearity.
        VIF > 5 for a variable indicates that it is highly collinear with the
        other input variables.
        """
        vif_df = pd.DataFrame()
        vif_df["Features"] = self.xvar_names
        vif_df["VIF Factor"] = [
            variance_inflation_factor(self.xvar, i) for i in range(self.xvar.shape[1])
        ]

        print(vif_df.sort_values("VIF Factor").round(2))

    def __cooks_dist_line(self, factor):
        """
        Helper function for plotting Cook's distance curves
        """
        p = self.nparams
        formula = lambda x: np.sqrt((factor * p * (1 - x)) / x)
        x = np.linspace(0.001, max(self.leverage), 50)
        y = formula(x)
        return x, y

In [ ]:
# plot diagnostics for the model
cls = Linear_Reg_Diagnostic(results_c_eur_area_skater)
fig, ax = cls()

In [ ]:
# test = results_c_eur_area_skater.outlier_test()
# print('Bad data points (bonf(p) < 0.05):')
# test[test['bonf(p)'] < 0.05]

In [ ]:
# outliers = test[test['bonf(p)'] < 0.05].index.values
# outliers_sum=list(outliers)
# all_data.loc[list(outliers)]

### 2.3 Difference in Difference validation


In [ ]:
# copy data for the model (DID validation)
data_aveiro_skater_DID = data_aveiro_skater.copy()

In [ ]:
data_aveiro_skater_DID.columns

In [ ]:
data_aveiro_skater_DID["Year"].unique()

In [ ]:
data_aveiro_skater_DID = data_aveiro_skater_DID[data_aveiro_skater_DID["T"] == 0]

In [ ]:
data_aveiro_skater_DID["Year"].unique()

In [ ]:
# check number of dwellings per cluster
pd.pivot_table(
    data_aveiro_skater_DID,
    values="Log_P_A",
    index=["Year"],
    aggfunc=lambda x: len(x.unique()),
)

In [ ]:
# Dummy for year on the DT zone
for i in data_aveiro_skater_DID["Year"].unique():
    data_aveiro_skater_DID["Year" + str(i)] = np.where(
        data_aveiro_skater_DID["Year"] == i, data_aveiro_skater_DID["D"], 0
    )

In [ ]:
data_aveiro_skater_DID.columns

In [ ]:
data_aveiro_skater_DID.head()

In [ ]:
# number of dwellings per cluster
pd.pivot_table(
    data_aveiro_skater_DID,
    values="Log_P_A",
    index=["Zona_SKATER"],
    aggfunc=lambda x: len(x.unique()),
).head(10)

In [ ]:
# get dummies for skater zones
data_aveiro_skater_DID = pd.get_dummies(
    data_aveiro_skater_DID, columns=["Zona_SKATER"], drop_first=True, dtype=float
)

In [ ]:
data_aveiro_skater_DID.columns

In [ ]:
# define dependent variable and independent variables

DID_y_eur_area_skater = data_aveiro_skater_DID["Log_P_A"].astype(float)

DID_X_eur_area_skater = data_aveiro_skater_DID[
    [
        "Year2006.0",
        "Year2007.0",
        "Year2008.0",
        "Year2009.0",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "Zona_SKATER_1.0",
        "Zona_SKATER_2.0",
        "Zona_SKATER_3.0",
        "Zona_SKATER_4.0",
        "Zona_SKATER_5.0",
        "Zona_SKATER_6.0",
        "Zona_SKATER_7.0",
        "Zona_SKATER_8.0",
        "Tot_AL",
        "IPI",
        "TAA",
    ]
].astype(float)

In [ ]:
# run the model
DID_X_eur_area_skater = sm.add_constant(DID_X_eur_area_skater)
model_DID_eur_area_skater = sm.OLS(DID_y_eur_area_skater, DID_X_eur_area_skater)
results_DID_eur_area_skater = model_DID_eur_area_skater.fit()
DID_skater_save = results_DID_eur_area_skater.summary2()
results_DID_eur_area_skater.summary2()

In [ ]:
# # export results to csv
# DID_skater_save.tables[0].to_csv('DID_skater_0.csv')
# DID_skater_save.tables[1].to_csv('DID_skater_1.csv')

In [ ]:
# plot diagnostics for the model
cls = Linear_Reg_Diagnostic(results_DID_eur_area_skater)
fig, ax = cls()

In [ ]:
# test = results_DID_eur_area_skater.outlier_test()
# print('Bad data points (bonf(p) < 0.05):')
# test[test['bonf(p)'] < 0.05]

In [ ]:
# outliers = test[test['bonf(p)'] < 0.05].index.values
# outliers_sum=outliers_sum+list(outliers)
# all_data.loc[list(outliers)]

### 2.4 Spatial Linear Regression (focused in Aveiro Center - SKATER Cluster)


In [ ]:
data_aveiro_skater_Robust = data_aveiro_skater.copy()

In [ ]:
data_aveiro_skater_Robust.columns

In [ ]:
data_aveiro_skater_Robust.shape

In [ ]:
# Geração da matriz W, a partir do subset, usando a de matriz de contiguidade Queen
w_Queen_skater = weights.contiguity.Queen.from_dataframe(data_aveiro_skater_Robust)
# Standardização das linhas
w_Queen_skater.transform = "R"

In [ ]:
# Histograma resultante da metodologia matriz Queen para pesos espaciais
pd.Series(w_Queen_skater.cardinalities).hist()

In [ ]:
w_Queen_skater.mean_neighbors

In [ ]:
# # image saved - this function is heavy and slows down the notebook
# plot_spatial_weights(w_Queen_skater, data_aveiro_skater_Robust)

In [ ]:
ax = data_aveiro_skater_Robust.plot(
    column="Zona_SKATER", categorical=True, legend=True, figsize=(10, 10), cmap="tab20c"
)
cx.add_basemap(
    ax, crs=data_aveiro_skater_Robust.crs, source=cx.providers.OpenStreetMap.Mapnik
)

In [ ]:
# apply above list to data
data_aveiro_skater_Robust["D"] = np.where(
    (data_aveiro_skater_Robust["Zona_SKATER"] == 3.0)
    | (data_aveiro_skater_Robust["Zona_SKATER"] == 6.0)
    | (data_aveiro_skater_Robust["Zona_SKATER"] == 7.0),
    1,
    0,
)

In [ ]:
data_aveiro_skater_Robust["D"].value_counts()

In [ ]:
sum(data_aveiro_skater_Robust["D"].value_counts())

In [ ]:
# calculate DT (true when both D and T are equal to 1)
data_aveiro_skater_Robust["DT"] = (
    data_aveiro_skater_Robust["D"] * data_aveiro_skater_Robust["T"]
)

In [ ]:
data_aveiro_skater_Robust["DT"].value_counts()

In [ ]:
(data_aveiro_skater_Robust["Log_P_A"]).describe()

In [ ]:
np.exp(0.367561)

In [ ]:
from pysal.explore import esda

moran = esda.moran.Moran(data_aveiro_skater_Robust["Log_P_A"], w_Queen_skater)

In [ ]:
moran.I

In [ ]:
moran.p_sim

In [ ]:
moran_l = esda.moran.Moran_Local(data_aveiro_skater_Robust["Log_P_A"], w_Queen_skater)

In [ ]:
from pysal.viz import splot

In [ ]:
figura, ax = moran_scatterplot(moran_l, p=0.05)
ax.set_title("Moran Scatterplot")
ax.set_xlabel("mediana_preço_hab")
ax.set_ylabel("Spatial Lag of mediana_preço_hab")
plt.show()

In [ ]:
data_aveiro_skater_Robust.columns

In [ ]:
data_aveiro_skater_Robust["MCA_1_lag"] = weights.lag_spatial(
    w_Queen_skater, data_aveiro_skater_Robust["MCA_1"]
)
data_aveiro_skater_Robust["MCA_2_lag"] = weights.lag_spatial(
    w_Queen_skater, data_aveiro_skater_Robust["MCA_2"]
)
data_aveiro_skater_Robust["MCA_3_lag"] = weights.lag_spatial(
    w_Queen_skater, data_aveiro_skater_Robust["MCA_3"]
)
data_aveiro_skater_Robust["MCA_4_lag"] = weights.lag_spatial(
    w_Queen_skater, data_aveiro_skater_Robust["MCA_4"]
)
data_aveiro_skater_Robust["Tot_AL_lag"] = weights.lag_spatial(
    w_Queen_skater, data_aveiro_skater_Robust["Tot_AL"]
)
data_aveiro_skater_Robust["Log_P_A_lag"] = weights.lag_spatial(
    w_Queen_skater, data_aveiro_skater_Robust["Log_P_A"]
)

In [ ]:
# get dummies for skater zones
data_aveiro_skater_Robust = pd.get_dummies(
    data_aveiro_skater_Robust, columns=["Zona_SKATER"], drop_first=True, dtype=float
)

In [ ]:
data_aveiro_skater_Robust.columns

In [ ]:
# Criação de dataframe com variável dependente, para uso nos modelos
Dep_Var_SKATER = data_aveiro_skater_Robust["Log_P_A"].astype(float)

# Criação de dataframe com variáveis independente, para uso nos modelos
Ind_Var_SKATER = data_aveiro_skater_Robust[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "TAA",
        "IPI",
        "Tot_AL",
        "Zona_SKATER_1.0",
        "Zona_SKATER_2.0",
        "Zona_SKATER_3.0",
        "Zona_SKATER_4.0",
        "Zona_SKATER_5.0",
        "Zona_SKATER_6.0",
    ]
].astype(float)

Ind_Var_lag_SKATER_lagX = data_aveiro_skater_Robust[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "TAA",
        "IPI",
        "Tot_AL",
        "Zona_SKATER_1.0",
        "Zona_SKATER_2.0",
        "Zona_SKATER_3.0",
        "Zona_SKATER_4.0",
        "Zona_SKATER_5.0",
        "Zona_SKATER_6.0",
        "MCA_1_lag",
        "MCA_2_lag",
        "MCA_3_lag",
        "MCA_4_lag",
        "Tot_AL_lag",
    ]
].astype(float)

Ind_Var_lag_SKATER_lagY = data_aveiro_skater_Robust[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "TAA",
        "IPI",
        "Tot_AL",
        "Zona_SKATER_1.0",
        "Zona_SKATER_2.0",
        "Zona_SKATER_3.0",
        "Zona_SKATER_4.0",
        "Zona_SKATER_5.0",
        "Zona_SKATER_6.0",
        "Log_P_A_lag",
    ]
].astype(float)

Ind_Var_lag_SKATER_lagXY = data_aveiro_skater_Robust[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "TAA",
        "IPI",
        "Tot_AL",
        "Zona_SKATER_1.0",
        "Zona_SKATER_2.0",
        "Zona_SKATER_3.0",
        "Zona_SKATER_4.0",
        "Zona_SKATER_5.0",
        "Zona_SKATER_6.0",
        "Log_P_A_lag",
        "MCA_1_lag",
        "MCA_2_lag",
        "MCA_3_lag",
        "MCA_4_lag",
        "Tot_AL_lag",
    ]
].astype(float)

In [ ]:
M_OLS_skater = spreg.OLS(
    Dep_Var_SKATER.values,  # Dependent variable
    Ind_Var_SKATER.values,  # Independent variable
    name_y="Log_P_A",  # Dependent variable name
    name_x=list(Ind_Var_SKATER.columns),  # Independent variable name
    w=w_Queen_skater,
    spat_diag=True,
    moran=True,
    name_w="w_Queen",
)

print(M_OLS_skater.summary)

In [ ]:
data_aveiro_skater_Robust["residuals_OLS"] = M_OLS_skater.u

In [ ]:
sns.displot(data_aveiro_skater_Robust["residuals_OLS"], bins=30, kde=True)
plt.title("Distributions of residuals")

In [ ]:
# standardisation of the residuals (Z-scores)
data_aveiro_skater_Robust["Z_Score_residuals_OLS"] = stats.zscore(
    data_aveiro_skater_Robust["residuals_OLS"]
)

In [ ]:
# distribution of the data against the expected normal distribution.
qqplot(data_aveiro_skater_Robust["Z_Score_residuals_OLS"], line="s")
plt.title("Normal Q-Q plot of residuals")

In [ ]:
# Create a Map - Equal Intervals
f, ax = plt.subplots(1, figsize=(8, 12))
ax = data_aveiro_skater_Robust.plot(
    column="Z_Score_residuals_OLS",  # Data to plot
    scheme="EqualInterval",  # Classification scheme
    cmap="bwr",  # Color palette
    edgecolor="k",  # Borderline color
    linewidth=0.1,  # Borderline width
    legend=True,  # Add legend
    legend_kwds={"fmt": "{:.1f}"},  # Remove decimals in legend (for legibility)
    k=10,
    ax=ax,
)

ax.set_title("residuals_OLS")
ax.set_axis_off()

In [ ]:
# Create a Map - Equal Intervals
f, ax = plt.subplots(1, figsize=(8, 12))
ax = data_aveiro_skater_Robust.plot(
    column="Z_Score_residuals_OLS",  # Data to plot
    scheme="StdMean",  # Classification scheme
    cmap="bwr",  # Color palette
    edgecolor="k",  # Borderline color
    linewidth=0.1,  # Borderline width
    legend=True,  # Add legend
    legend_kwds={
        "fmt": "{:.1f}",
        "loc": "lower right",
    },  # Remove decimals in legend (for legibility)
    ax=ax,
)

ax.set_title("residuals_OLS")
ax.set_axis_off()

## 3 Linear Regressions (Aveiro) - FR (Freguesias)


### 3.1 Data Preparation for DID Linear Regression


In [ ]:
# transform data to geodataframe
data_aveiro_fr = gpd.GeoDataFrame(all_data_fr, geometry="geometry")

In [ ]:
data_aveiro_fr.columns

In [ ]:
# check number of dwellings per cluster
pd.pivot_table(
    data_aveiro_fr,
    values="Cluster_LP",
    index=["FR11"],
    aggfunc=lambda x: len(x.unique()),
).head(10)

In [ ]:
data_aveiro_fr.dtypes

In [ ]:
data_aveiro_fr.head()

In [ ]:
ax = data_aveiro_fr.plot(
    column="FR11", categorical=True, legend=True, figsize=(10, 10), cmap="tab20"
)
cx.add_basemap(ax, crs=data_aveiro_fr.crs, source=cx.providers.OpenStreetMap.Mapnik)

In [ ]:
# apply above list to data
data_aveiro_fr["D"] = np.where((data_aveiro_fr["FR11"] == 12), 1, 0)

In [ ]:
data_aveiro_fr["D"].value_counts()

In [ ]:
# check areas defined as control and intervention areas (D=0 and D=1)
ax = data_aveiro_fr.plot(
    column=data_aveiro_fr["D"],
    categorical=True,
    legend=True,
    figsize=(10, 10),
    cmap="tab20",
)
cx.add_basemap(ax, crs=data_aveiro_fr.crs, source=cx.providers.OpenStreetMap.Mapnik)

### 3.2 Linear Regression (focused in Aveiro Center - Freguesias)


In [ ]:
# calculate DT (true when both D and T are equal to 1)
data_aveiro_fr["DT"] = data_aveiro_fr["D"] * data_aveiro_fr["T"]

In [ ]:
data_aveiro_fr["DT"].value_counts()

In [ ]:
data_aveiro_fr["D"].value_counts()

In [ ]:
# get dummies for skater zones
data_aveiro_fr_ols = pd.get_dummies(
    data_aveiro_fr, columns=["FR11"], drop_first=True, dtype=float
)

In [ ]:
data_aveiro_fr_ols.columns

In [ ]:
# define X and y
c_y_eur_area_fr = data_aveiro_fr_ols["Log_P_A"].astype(float)

c_X_eur_area_fr = data_aveiro_fr_ols[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "IPI",
        "Tot_AL",
        "TAA",
        "FR11_5",
        "FR11_6",
        "FR11_10",
        "FR11_12",
        "FR11_13",
    ]
].astype(float)

# removido IPI - VIF elevado

In [ ]:
# linear model for casasapo with log of price per square meter as dependent variable
c_X_eur_area_fr = sm.add_constant(c_X_eur_area_fr)
model_c_eur_area_fr = sm.OLS(c_y_eur_area_fr, c_X_eur_area_fr)
results_c_eur_area_fr = model_c_eur_area_fr.fit()
fr_save = results_c_eur_area_fr.summary2()
results_c_eur_area_fr.summary2()

In [ ]:
# # export results to csv
# fr_save.tables[0].to_csv('fr_0.csv')
# fr_save.tables[1].to_csv('fr_1.csv')

In [ ]:
# plot diagnostics for the model
cls = Linear_Reg_Diagnostic(results_c_eur_area_fr)
fig, ax = cls()

In [ ]:
# test = results_c_eur_area_fr.outlier_test()
# print('Bad data points (bonf(p) < 0.05):')
# test[test['bonf(p)'] < 0.05]

In [ ]:
# outliers = test[test['bonf(p)'] < 0.05].index.values
# outliers_sum=outliers_sum+list(outliers)
# all_data.loc[list(outliers)]

### 3.3 Difference in Difference validation


In [ ]:
# copy data for the model (DID validation)
data_aveiro_fr_DID = data_aveiro_fr.copy()

In [ ]:
data_aveiro_fr_DID.columns

In [ ]:
data_aveiro_fr_DID["Year"].unique()

In [ ]:
data_aveiro_fr_DID = data_aveiro_fr_DID[data_aveiro_fr_DID["T"] == 0]
data_aveiro_fr_DID["Year"].unique()

In [ ]:
for i in data_aveiro_fr_DID["Year"].unique():
    data_aveiro_fr_DID["Year" + str(i)] = np.where(
        data_aveiro_fr_DID["Year"] == i, data_aveiro_fr_DID["D"], 0
    )

In [ ]:
data_aveiro_fr_DID.head()

In [ ]:
# number of dwellings per cluster
pd.pivot_table(
    data_aveiro_fr_DID,
    values="Log_P_A",
    index=["FR11"],
    aggfunc=lambda x: len(x.unique()),
).head(10)
# não incluir cluster 102 e 106, por escassez de dados

In [ ]:
data_aveiro_fr_DID.columns

In [ ]:
# get dummies for FR11 zones
data_aveiro_fr_DID = pd.get_dummies(
    data_aveiro_fr_DID, columns=["FR11"], drop_first=True, dtype=float
)

In [ ]:
data_aveiro_fr_DID.columns

In [ ]:
# define dependent variable and independent variables
DID_y_eur_area_fr = data_aveiro_fr_DID["Log_P_A"].astype(float)

DID_X_eur_area_fr = data_aveiro_fr_DID[
    [
        "Year2006.0",
        "Year2007.0",
        "Year2008.0",
        "Year2009.0",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "Tot_AL",
        "IPI",
        "TAA",
        "FR11_5",
        "FR11_6",
        "FR11_10",
        "FR11_12",
        "FR11_13",
    ]
].astype(float)
#

In [ ]:
# run the model
DID_X_eur_area_fr = sm.add_constant(DID_X_eur_area_fr)
model_DID_eur_area_fr = sm.OLS(DID_y_eur_area_fr, DID_X_eur_area_fr)
results_DID_eur_area_fr = model_DID_eur_area_fr.fit()
DID_fr_save = results_DID_eur_area_fr.summary2()
results_DID_eur_area_fr.summary2()

In [ ]:
# # export results to csv
# DID_fr_save.tables[0].to_csv('DID_fr_0.csv')
# DID_fr_save.tables[1].to_csv('DID_fr_1.csv')


In [ ]:
# plot diagnostics for the model
cls = Linear_Reg_Diagnostic(results_DID_eur_area_fr)
fig, ax = cls()

In [ ]:
# test = results_DID_eur_area_fr.outlier_test()
# print('Bad data points (bonf(p) < 0.05):')
# test[test['bonf(p)'] < 0.05]

In [ ]:
# outliers = test[test['bonf(p)'] < 0.05].index.values
# outliers_sum=outliers_sum+list(outliers)
# all_data.loc[list(outliers)]

### 3.4 Spatial Linear Regression (focused in Aveiro Center - FR11)


In [ ]:
data_aveiro_fr_Robust = data_aveiro_fr.copy()

In [ ]:
data_aveiro_fr_Robust.columns

In [ ]:
data_aveiro_fr_Robust.shape

In [ ]:
# Geração da matriz W, a partir do subset, usando a de matriz de contiguidade Queen
w_Queen_fr = weights.contiguity.Queen.from_dataframe(data_aveiro_fr_Robust)
# Standardização das linhas
w_Queen_fr.transform = "R"

In [ ]:
# Histograma resultante da metodologia matriz Queen para pesos espaciais
pd.Series(w_Queen_fr.cardinalities).hist()

In [ ]:
w_Queen_fr.mean_neighbors

In [ ]:
# # image saved - this function is heavy and slows down the notebook
# plot_spatial_weights(w_Queen_fr, data_aveiro_fr_Robust)

In [ ]:
ax = data_aveiro_fr_Robust.plot(
    column="FR11", categorical=True, legend=True, figsize=(10, 10), cmap="tab20c"
)
cx.add_basemap(
    ax, crs=data_aveiro_fr_Robust.crs, source=cx.providers.OpenStreetMap.Mapnik
)

In [ ]:
# apply above list to data
data_aveiro_fr_Robust["D"] = np.where((data_aveiro_fr_Robust["FR11"] == 12), 1, 0)

In [ ]:
data_aveiro_fr_Robust["D"].value_counts()

In [ ]:
sum(data_aveiro_fr_Robust["D"].value_counts())

In [ ]:
# calculate DT (true when both D and T are equal to 1)
data_aveiro_fr_Robust["DT"] = data_aveiro_fr_Robust["D"] * data_aveiro_fr_Robust["T"]

In [ ]:
data_aveiro_fr_Robust["DT"].value_counts()

In [ ]:
data_aveiro_fr_Robust.columns

In [ ]:
data_aveiro_fr_Robust["MCA_1_lag"] = weights.lag_spatial(
    w_Queen_fr, data_aveiro_fr_Robust["MCA_1"]
)
data_aveiro_fr_Robust["MCA_2_lag"] = weights.lag_spatial(
    w_Queen_fr, data_aveiro_fr_Robust["MCA_2"]
)
data_aveiro_fr_Robust["MCA_3_lag"] = weights.lag_spatial(
    w_Queen_fr, data_aveiro_fr_Robust["MCA_3"]
)
data_aveiro_fr_Robust["MCA_4_lag"] = weights.lag_spatial(
    w_Queen_fr, data_aveiro_fr_Robust["MCA_4"]
)
data_aveiro_fr_Robust["Tot_AL_lag"] = weights.lag_spatial(
    w_Queen_fr, data_aveiro_fr_Robust["Tot_AL"]
)
data_aveiro_fr_Robust["Log_P_A_lag"] = weights.lag_spatial(
    w_Queen_skater, data_aveiro_fr_Robust["Log_P_A"]
)

In [ ]:
# get dummies for skater zones
data_aveiro_fr_Robust = pd.get_dummies(
    data_aveiro_fr_Robust, columns=["FR11"], drop_first=True, dtype=float
)

In [ ]:
data_aveiro_fr_Robust.columns

In [ ]:
# Criação de dataframe com variável dependente, para uso nos modelos
Dep_Var_FR = data_aveiro_fr_Robust["Log_P_A"].astype(float)

# Criação de dataframe com variáveis independente, para uso nos modelos
Ind_Var_FR = data_aveiro_fr_Robust[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "TAA",
        "IPI",
        "Tot_AL",
        "FR11_5",
        "FR11_6",
        "FR11_10",
        "FR11_12",
        "FR11_13",
    ]
].astype(float)

Ind_Var_FR_lagX = data_aveiro_fr_Robust[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "TAA",
        "IPI",
        "Tot_AL",
        "FR11_5",
        "FR11_6",
        "FR11_10",
        "FR11_12",
        "FR11_13",
        "MCA_1_lag",
        "MCA_2_lag",
        "MCA_3_lag",
        "MCA_4_lag",
        "Tot_AL_lag",
    ]
].astype(float)

Ind_Var_FR_lagY = data_aveiro_fr_Robust[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "TAA",
        "IPI",
        "Tot_AL",
        "FR11_5",
        "FR11_6",
        "FR11_10",
        "FR11_12",
        "FR11_13",
        "Log_P_A_lag",
    ]
].astype(float)

Ind_Var_FR_lagXY = data_aveiro_fr_Robust[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "TAA",
        "IPI",
        "Tot_AL",
        "FR11_5",
        "FR11_6",
        "FR11_10",
        "FR11_12",
        "FR11_13",
        "Log_P_A_lag",
        "MCA_1_lag",
        "MCA_2_lag",
        "MCA_3_lag",
        "MCA_4_lag",
        "Tot_AL_lag",
    ]
].astype(float)

In [ ]:
M_OLS_FR = spreg.OLS(
    Dep_Var_FR.values,  # Dependent variable
    Ind_Var_FR.values,  # Independent variable
    name_y="Log_P_A",  # Dependent variable name
    name_x=list(Ind_Var_FR.columns),  # Independent variable name
    w=w_Queen_fr,
    spat_diag=True,
    moran=True,
    name_w="w_Queen",
)

print(M_OLS_FR.summary)

In [ ]:
data_aveiro_fr_Robust["residuals_OLS"] = M_OLS_FR.u

In [ ]:
sns.displot(data_aveiro_fr_Robust["residuals_OLS"], bins=30, kde=True)
plt.title("Distributions of residuals")

In [ ]:
# standardisation of the residuals (Z-scores)
data_aveiro_fr_Robust["Z_Score_residuals_OLS"] = stats.zscore(
    data_aveiro_fr_Robust["residuals_OLS"]
)

In [ ]:
# distribution of the data against the expected normal distribution.
qqplot(data_aveiro_fr_Robust["Z_Score_residuals_OLS"], line="s")
plt.title("Normal Q-Q plot of residuals")

In [ ]:
# Create a Map - Equal Intervals
f, ax = plt.subplots(1, figsize=(8, 12))
ax = data_aveiro_fr_Robust.plot(
    column="Z_Score_residuals_OLS",  # Data to plot
    scheme="EqualInterval",  # Classification scheme
    cmap="bwr",  # Color palette
    edgecolor="k",  # Borderline color
    linewidth=0.1,  # Borderline width
    legend=True,  # Add legend
    legend_kwds={"fmt": "{:.1f}"},  # Remove decimals in legend (for legibility)
    k=10,
    ax=ax,
)

ax.set_title("residuals_OLS")
ax.set_axis_off()

In [ ]:
# Create a Map - Equal Intervals
f, ax = plt.subplots(1, figsize=(8, 12))
ax = data_aveiro_fr_Robust.plot(
    column="Z_Score_residuals_OLS",  # Data to plot
    scheme="StdMean",  # Classification scheme
    cmap="bwr",  # Color palette
    edgecolor="k",  # Borderline color
    linewidth=0.1,  # Borderline width
    legend=True,  # Add legend
    legend_kwds={
        "fmt": "{:.1f}",
        "loc": "lower right",
    },  # Remove decimals in legend (for legibility)
    ax=ax,
)

ax.set_title("residuals_OLS")
ax.set_axis_off()

## 4 Linear Regressions (Aveiro) - LP


### 4.1 Data Preparation for DID Linear Regression


In [ ]:
# transform data to geodataframe
data_aveiro_LP = gpd.GeoDataFrame(all_data_LP, geometry="geometry")

In [ ]:
data_aveiro_LP

In [ ]:
data_aveiro_LP.columns

In [ ]:
# view Zona_SKATER clusters
ax = data_aveiro_LP.plot(
    figsize=(10, 20),
    column=data_aveiro_LP["Cluster_LP"],
    categorical=True,
    edgecolor="w",
    legend=False,
    linewidth=0.2,
    cmap="tab20",
)
cx.add_basemap(ax, crs=data_aveiro_LP.crs, source=cx.providers.OpenStreetMap.Mapnik)

In [ ]:
# Area de Intervenção, a ser considerada D = 1
lista_D1 = [
    19,
    22,
    31,
    40,
    41,
    42,
    80,
    102,
    105,
    107,
    109,
    110,
    156,
    159,
    160,
    161,
    165,
    166,
]

In [ ]:
# lista de zonas na fronteira D1/D0:
# 26, 57, 101, 103, 104, 111, 155, 161, 165

In [ ]:
data_aveiro_LP["D"] = np.where(data_aveiro_LP["Cluster_LP"].isin(lista_D1), 1, 0)

In [ ]:
# view Zona_SKATER clusters
ax = data_aveiro_LP.plot(
    figsize=(10, 10),
    column=data_aveiro_LP["D"],
    linewidth=0.2,
    categorical=True,
    legend=True,
    cmap="tab20",
)
cx.add_basemap(ax, crs=data_aveiro_LP.crs, source=cx.providers.OpenStreetMap.Mapnik)

In [ ]:
data_aveiro_LP.head()

In [ ]:
# check no. of dwellings per cluster in Aveiro Center
pd.pivot_table(
    data_aveiro_LP,
    values="Log_P_A",
    index=["Cluster_LP"],
    aggfunc=lambda x: len(x.unique()),
).head(10)
# não incluir cluster 80, 102 e 106, por escassez de dados

In [ ]:
data_aveiro_LP.to_pickle("../Data/data_aveiro_LP.piclo")

In [ ]:
data_aveiro_LP["D"].value_counts()

In [ ]:
sum(data_aveiro_LP["D"].value_counts())

In [ ]:
# calculate DT (true when both D and T are equal to 1)
data_aveiro_LP["DT"] = data_aveiro_LP["D"] * data_aveiro_LP["T"]

In [ ]:
data_aveiro_LP["DT"].value_counts()

In [ ]:
# check log_P_A distribution in the territory
ax = data_aveiro_LP.plot(
    column=data_aveiro_LP["Log_P_A"], legend=True, figsize=(10, 10), cmap="viridis"
)
cx.add_basemap(ax, crs=data_aveiro_LP.crs, source=cx.providers.OpenStreetMap.Mapnik)

### 4.2 Linear Regression (focused in Aveiro Center - LP)


In [ ]:
# check number of dwellings per cluster
pd.pivot_table(
    data_aveiro_LP,
    values="Log_P_A",
    index=["Cluster_LP"],
    aggfunc=lambda x: len(x.unique()),
).head(10)

In [ ]:
# check number of dwellings per cluster
pd.pivot_table(
    data_aveiro_LP, values="Cluster_LP", index=["D"], aggfunc=lambda x: len(x.unique())
)

In [ ]:
data_aveiro_LP.columns

In [ ]:
# get dummies for LP zones
data_aveiro_LP_ols = pd.get_dummies(
    data_aveiro_LP, columns=["Cluster_LP"], drop_first=True, dtype=float
)

In [ ]:
data_aveiro_LP_ols.columns

In [ ]:
# define X and y
c_y_eur_area_LP = data_aveiro_LP_ols["Log_P_A"].astype(float)

c_X_eur_area_LP = data_aveiro_LP_ols[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "IPI",
        "Tot_AL",
        "TAA",
        "Cluster_LP_5.0",
        "Cluster_LP_13.0",
        "Cluster_LP_14.0",
        "Cluster_LP_18.0",
        "Cluster_LP_19.0",
        "Cluster_LP_20.0",
        "Cluster_LP_21.0",
        "Cluster_LP_22.0",
        "Cluster_LP_24.0",
        "Cluster_LP_25.0",
        "Cluster_LP_26.0",
        "Cluster_LP_27.0",
        "Cluster_LP_28.0",
        "Cluster_LP_29.0",
        "Cluster_LP_30.0",
        "Cluster_LP_31.0",
        "Cluster_LP_37.0",
        "Cluster_LP_38.0",
        "Cluster_LP_39.0",
        "Cluster_LP_40.0",
        "Cluster_LP_41.0",
        "Cluster_LP_42.0",
        "Cluster_LP_43.0",
        "Cluster_LP_45.0",
        "Cluster_LP_57.0",
        "Cluster_LP_58.0",
        "Cluster_LP_59.0",
        "Cluster_LP_60.0",
        "Cluster_LP_63.0",
        "Cluster_LP_64.0",
        "Cluster_LP_65.0",
        "Cluster_LP_66.0",
        "Cluster_LP_67.0",
        "Cluster_LP_80.0",
        "Cluster_LP_82.0",
        "Cluster_LP_85.0",
        "Cluster_LP_89.0",
        "Cluster_LP_90.0",
        "Cluster_LP_91.0",
        "Cluster_LP_100.0",
        "Cluster_LP_101.0",
        "Cluster_LP_102.0",
        "Cluster_LP_103.0",
        "Cluster_LP_104.0",
        "Cluster_LP_105.0",
        "Cluster_LP_107.0",
        "Cluster_LP_108.0",
        "Cluster_LP_109.0",
        "Cluster_LP_110.0",
        "Cluster_LP_111.0",
        "Cluster_LP_124.0",
        "Cluster_LP_125.0",
        "Cluster_LP_129.0",
        "Cluster_LP_150.0",
        "Cluster_LP_151.0",
        "Cluster_LP_152.0",
        "Cluster_LP_153.0",
        "Cluster_LP_154.0",
        "Cluster_LP_155.0",
        "Cluster_LP_156.0",
        "Cluster_LP_157.0",
        "Cluster_LP_158.0",
        "Cluster_LP_159.0",
        "Cluster_LP_160.0",
        "Cluster_LP_161.0",
        "Cluster_LP_162.0",
        "Cluster_LP_163.0",
        "Cluster_LP_164.0",
        "Cluster_LP_165.0",
        "Cluster_LP_166.0",
    ]
].astype(float)

In [ ]:
# linear model for casasapo with log of price per square meter as dependent variable
c_X_eur_area_LP = sm.add_constant(c_X_eur_area_LP)
model_c_eur_area_LP = sm.OLS(c_y_eur_area_LP, c_X_eur_area_LP)
results_c_eur_area_LP = model_c_eur_area_LP.fit()
lp_save = results_c_eur_area_LP.summary2()
results_c_eur_area_LP.summary2()

In [ ]:
# # export results to csv
# lp_save.tables[0].to_csv('lp_0.csv')
# lp_save.tables[1].to_csv('lp_1.csv')


In [ ]:
# plot diagnostics for the model
cls = Linear_Reg_Diagnostic(results_c_eur_area_LP)
fig, ax = cls()

In [ ]:
# test = results_c_eur_area_LP.outlier_test()
# print('Bad data points (bonf(p) < 0.05):')
# test[test['bonf(p)'] < 0.05]

In [ ]:
# outliers = test[test['bonf(p)'] < 0.05].index.values
# outliers_sum=outliers_sum+list(outliers)
# all_data.loc[list(outliers)]

### 4.3 Difference in Difference validation


In [ ]:
# copy data for the model (DID validation)
data_aveiro_LP_DID = data_aveiro_LP.copy()

In [ ]:
data_aveiro_LP_DID.columns

In [ ]:
data_aveiro_LP_DID["Year"].unique()

In [ ]:
data_aveiro_LP_DID = data_aveiro_LP_DID[data_aveiro_LP_DID["T"] == 0]

In [ ]:
data_aveiro_LP_DID["Year"].unique()

In [ ]:
for i in data_aveiro_LP_DID["Year"].unique():
    data_aveiro_LP_DID["Year" + str(i)] = np.where(
        data_aveiro_LP_DID["Year"] == i, data_aveiro_LP_DID["D"], 0
    )

In [ ]:
data_aveiro_LP_DID.head()

In [ ]:
# number of dwellings per cluster
pd.pivot_table(
    data_aveiro_LP_DID,
    values="Log_P_A",
    index=["Cluster_LP"],
    aggfunc=lambda x: len(x.unique()),
).head(10)
# não incluir cluster 102 e 106, por escassez de dados

In [ ]:
data_aveiro_LP_DID.columns

In [ ]:
# get dummies for LP zones
data_aveiro_LP_DID = pd.get_dummies(
    data_aveiro_LP_DID, columns=["Cluster_LP"], drop_first=True, dtype=float
)

In [ ]:
data_aveiro_LP_DID.columns

In [ ]:
# define dependent variable and independent variables
DID_y_eur_area_LP = data_aveiro_LP_DID["Log_P_A"].astype(float)

DID_X_eur_area_LP = data_aveiro_LP_DID[
    [
        "Year2006.0",
        "Year2007.0",
        "Year2008.0",
        "Year2009.0",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "IPI",
        "Tot_AL",
        "TAA",
        "Cluster_LP_5.0",
        "Cluster_LP_13.0",
        "Cluster_LP_14.0",
        "Cluster_LP_18.0",
        "Cluster_LP_19.0",
        "Cluster_LP_20.0",
        "Cluster_LP_21.0",
        "Cluster_LP_22.0",
        "Cluster_LP_24.0",
        "Cluster_LP_25.0",
        "Cluster_LP_26.0",
        "Cluster_LP_27.0",
        "Cluster_LP_28.0",
        "Cluster_LP_29.0",
        "Cluster_LP_30.0",
        "Cluster_LP_31.0",
        "Cluster_LP_37.0",
        "Cluster_LP_38.0",
        "Cluster_LP_39.0",
        "Cluster_LP_40.0",
        "Cluster_LP_41.0",
        "Cluster_LP_42.0",
        "Cluster_LP_43.0",
        "Cluster_LP_45.0",
        "Cluster_LP_58.0",
        "Cluster_LP_59.0",
        "Cluster_LP_60.0",
        "Cluster_LP_63.0",
        "Cluster_LP_64.0",
        "Cluster_LP_65.0",
        "Cluster_LP_67.0",
        "Cluster_LP_82.0",
        "Cluster_LP_85.0",
        "Cluster_LP_89.0",
        "Cluster_LP_91.0",
        "Cluster_LP_100.0",
        "Cluster_LP_101.0",
        "Cluster_LP_104.0",
        "Cluster_LP_105.0",
        "Cluster_LP_109.0",
        "Cluster_LP_111.0",
        "Cluster_LP_124.0",
        "Cluster_LP_150.0",
        "Cluster_LP_151.0",
        "Cluster_LP_152.0",
        "Cluster_LP_153.0",
        "Cluster_LP_154.0",
        "Cluster_LP_155.0",
        "Cluster_LP_156.0",
        "Cluster_LP_157.0",
        "Cluster_LP_158.0",
        "Cluster_LP_159.0",
        "Cluster_LP_160.0",
        "Cluster_LP_161.0",
        "Cluster_LP_162.0",
        "Cluster_LP_163.0",
        "Cluster_LP_164.0",
        "Cluster_LP_166.0",
    ]
].astype(float)

In [ ]:
# run the model
DID_X_eur_area_LP = sm.add_constant(DID_X_eur_area_LP)
model_DID_eur_area_LP = sm.OLS(DID_y_eur_area_LP, DID_X_eur_area_LP)
results_DID_eur_area_LP = model_DID_eur_area_LP.fit()
DID_lp_save = results_DID_eur_area_LP.summary2()
results_DID_eur_area_LP.summary2()

In [ ]:
# # export results to csv
# DID_lp_save.tables[0].to_csv('DID_lp_0.csv')
# DID_lp_save.tables[1].to_csv('DID_lp_1.csv')


In [ ]:
# plot diagnostics for the model
cls = Linear_Reg_Diagnostic(results_DID_eur_area_LP)
fig, ax = cls()

In [ ]:
# test = results_DID_eur_area_LP.outlier_test()
# print('Bad data points (bonf(p) < 0.05):')
# test[test['bonf(p)'] < 0.05]

In [ ]:
# outliers = test[test['bonf(p)'] < 0.05].index.values
# outliers_sum=outliers_sum+list(outliers)
# all_data.loc[list(outliers)]

### 4.4 Spatial Linear Regression (focused in Aveiro Center - LP)


In [ ]:
data_aveiro_LP_Robust = data_aveiro_LP.copy()

In [ ]:
data_aveiro_LP_Robust.columns

In [ ]:
data_aveiro_LP_Robust.shape

In [ ]:
# Geração da matriz W, a partir do subset, usando a de matriz de contiguidade Queen
w_Queen_LP = weights.contiguity.Queen.from_dataframe(data_aveiro_LP_Robust)
# Standardização das linhas
w_Queen_LP.transform = "R"

In [ ]:
# Histograma resultante da metodologia matriz Queen para pesos espaciais
pd.Series(w_Queen_LP.cardinalities).hist()

In [ ]:
w_Queen_LP.mean_neighbors

In [ ]:
# image saved - this function is heavy and slows down the notebook
# plot_spatial_weights(w_Queen_LP, data_aveiro_LP_Robust)

In [ ]:
ax = data_aveiro_LP_Robust.plot(
    column="Cluster_LP", categorical=True, legend=True, figsize=(10, 20), cmap="tab20c"
)
cx.add_basemap(
    ax, crs=data_aveiro_LP_Robust.crs, source=cx.providers.OpenStreetMap.Mapnik
)

In [ ]:
data_aveiro_LP_Robust["D"] = np.where(
    data_aveiro_LP_Robust["Cluster_LP"].isin(lista_D1), 1, 0
)

In [ ]:
data_aveiro_LP_Robust["D"].value_counts()

In [ ]:
sum(data_aveiro_LP_Robust["D"].value_counts())

In [ ]:
# calculate DT (true when both D and T are equal to 1)
data_aveiro_LP_Robust["DT"] = data_aveiro_LP_Robust["D"] * data_aveiro_LP_Robust["T"]

In [ ]:
data_aveiro_LP_Robust["DT"].value_counts()

In [ ]:
sum(data_aveiro_LP_Robust["DT"].value_counts())

In [ ]:
data_aveiro_LP_Robust.columns

In [ ]:
# get dummies for skater zones
data_aveiro_LP_Robust = pd.get_dummies(
    data_aveiro_LP_Robust, columns=["Cluster_LP"], drop_first=True, dtype=float
)

In [ ]:
data_aveiro_LP_Robust.columns

In [ ]:
data_aveiro_LP_Robust["MCA_1_lag"] = weights.lag_spatial(
    w_Queen_LP, data_aveiro_LP_Robust["MCA_1"]
)

data_aveiro_LP_Robust["MCA_2_lag"] = weights.lag_spatial(
    w_Queen_LP, data_aveiro_LP_Robust["MCA_2"]
)

data_aveiro_LP_Robust["MCA_3_lag"] = weights.lag_spatial(
    w_Queen_LP, data_aveiro_LP_Robust["MCA_3"]
)

data_aveiro_LP_Robust["MCA_4_lag"] = weights.lag_spatial(
    w_Queen_LP, data_aveiro_LP_Robust["MCA_4"]
)

data_aveiro_LP_Robust["Tot_AL_lag"] = weights.lag_spatial(
    w_Queen_LP, data_aveiro_LP_Robust["Tot_AL"]
)

data_aveiro_LP_Robust["Log_P_A_lag"] = weights.lag_spatial(
    w_Queen_LP, data_aveiro_LP_Robust["Log_P_A"]
)

In [ ]:
data_aveiro_LP_Robust.columns

In [ ]:
# Criação de dataframe com variável dependente, para uso nos modelos
Dep_Var_LP = data_aveiro_LP_Robust["Log_P_A"].astype(float)

# Criação de dataframe com variáveis independente, para uso nos modelos
Ind_Var_LP = data_aveiro_LP_Robust[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "TAA",
        "IPI",
        "Tot_AL",
        "Cluster_LP_5.0",
        "Cluster_LP_13.0",
        "Cluster_LP_14.0",
        "Cluster_LP_18.0",
        "Cluster_LP_19.0",
        "Cluster_LP_20.0",
        "Cluster_LP_21.0",
        "Cluster_LP_22.0",
        "Cluster_LP_24.0",
        "Cluster_LP_25.0",
        "Cluster_LP_26.0",
        "Cluster_LP_27.0",
        "Cluster_LP_28.0",
        "Cluster_LP_29.0",
        "Cluster_LP_30.0",
        "Cluster_LP_31.0",
        "Cluster_LP_37.0",
        "Cluster_LP_38.0",
        "Cluster_LP_39.0",
        "Cluster_LP_40.0",
        "Cluster_LP_41.0",
        "Cluster_LP_42.0",
        "Cluster_LP_43.0",
        "Cluster_LP_45.0",
        "Cluster_LP_57.0",
        "Cluster_LP_58.0",
        "Cluster_LP_59.0",
        "Cluster_LP_60.0",
        "Cluster_LP_63.0",
        "Cluster_LP_64.0",
        "Cluster_LP_65.0",
        "Cluster_LP_66.0",
        "Cluster_LP_67.0",
        "Cluster_LP_80.0",
        "Cluster_LP_82.0",
        "Cluster_LP_85.0",
        "Cluster_LP_89.0",
        "Cluster_LP_90.0",
        "Cluster_LP_91.0",
        "Cluster_LP_100.0",
        "Cluster_LP_101.0",
        "Cluster_LP_102.0",
        "Cluster_LP_103.0",
        "Cluster_LP_104.0",
        "Cluster_LP_105.0",
        "Cluster_LP_107.0",
        "Cluster_LP_108.0",
        "Cluster_LP_109.0",
        "Cluster_LP_110.0",
        "Cluster_LP_111.0",
        "Cluster_LP_124.0",
        "Cluster_LP_125.0",
        "Cluster_LP_129.0",
        "Cluster_LP_150.0",
        "Cluster_LP_151.0",
        "Cluster_LP_152.0",
        "Cluster_LP_154.0",
        "Cluster_LP_155.0",
        "Cluster_LP_156.0",
        "Cluster_LP_157.0",
        "Cluster_LP_158.0",
        "Cluster_LP_159.0",
        "Cluster_LP_160.0",
        "Cluster_LP_161.0",
        "Cluster_LP_163.0",
        "Cluster_LP_164.0",
        "Cluster_LP_165.0",
        "Cluster_LP_166.0",
    ]
].astype(float)

Ind_Var_LP_lagX = data_aveiro_LP_Robust[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "TAA",
        "IPI",
        "Tot_AL",
        "Cluster_LP_5.0",
        "Cluster_LP_13.0",
        "Cluster_LP_14.0",
        "Cluster_LP_18.0",
        "Cluster_LP_19.0",
        "Cluster_LP_20.0",
        "Cluster_LP_21.0",
        "Cluster_LP_22.0",
        "Cluster_LP_24.0",
        "Cluster_LP_25.0",
        "Cluster_LP_26.0",
        "Cluster_LP_27.0",
        "Cluster_LP_28.0",
        "Cluster_LP_29.0",
        "Cluster_LP_30.0",
        "Cluster_LP_31.0",
        "Cluster_LP_37.0",
        "Cluster_LP_38.0",
        "Cluster_LP_39.0",
        "Cluster_LP_40.0",
        "Cluster_LP_41.0",
        "Cluster_LP_42.0",
        "Cluster_LP_43.0",
        "Cluster_LP_45.0",
        "Cluster_LP_57.0",
        "Cluster_LP_58.0",
        "Cluster_LP_59.0",
        "Cluster_LP_60.0",
        "Cluster_LP_63.0",
        "Cluster_LP_64.0",
        "Cluster_LP_65.0",
        "Cluster_LP_66.0",
        "Cluster_LP_67.0",
        "Cluster_LP_80.0",
        "Cluster_LP_82.0",
        "Cluster_LP_85.0",
        "Cluster_LP_89.0",
        "Cluster_LP_90.0",
        "Cluster_LP_91.0",
        "Cluster_LP_100.0",
        "Cluster_LP_101.0",
        "Cluster_LP_102.0",
        "Cluster_LP_103.0",
        "Cluster_LP_104.0",
        "Cluster_LP_105.0",
        "Cluster_LP_107.0",
        "Cluster_LP_108.0",
        "Cluster_LP_109.0",
        "Cluster_LP_110.0",
        "Cluster_LP_111.0",
        "Cluster_LP_124.0",
        "Cluster_LP_125.0",
        "Cluster_LP_129.0",
        "Cluster_LP_150.0",
        "Cluster_LP_151.0",
        "Cluster_LP_152.0",
        "Cluster_LP_154.0",
        "Cluster_LP_155.0",
        "Cluster_LP_156.0",
        "Cluster_LP_157.0",
        "Cluster_LP_158.0",
        "Cluster_LP_159.0",
        "Cluster_LP_160.0",
        "Cluster_LP_161.0",
        "Cluster_LP_163.0",
        "Cluster_LP_164.0",
        "Cluster_LP_165.0",
        "Cluster_LP_166.0",
        "MCA_1_lag",
        "MCA_2_lag",
        "MCA_3_lag",
        "MCA_4_lag",
        "Tot_AL_lag",
    ]
].astype(float)

Ind_Var_LP_lagY = data_aveiro_LP_Robust[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "TAA",
        "IPI",
        "Tot_AL",
        "Cluster_LP_5.0",
        "Cluster_LP_13.0",
        "Cluster_LP_14.0",
        "Cluster_LP_18.0",
        "Cluster_LP_19.0",
        "Cluster_LP_20.0",
        "Cluster_LP_21.0",
        "Cluster_LP_22.0",
        "Cluster_LP_24.0",
        "Cluster_LP_25.0",
        "Cluster_LP_26.0",
        "Cluster_LP_27.0",
        "Cluster_LP_28.0",
        "Cluster_LP_29.0",
        "Cluster_LP_30.0",
        "Cluster_LP_31.0",
        "Cluster_LP_37.0",
        "Cluster_LP_38.0",
        "Cluster_LP_39.0",
        "Cluster_LP_40.0",
        "Cluster_LP_41.0",
        "Cluster_LP_42.0",
        "Cluster_LP_43.0",
        "Cluster_LP_45.0",
        "Cluster_LP_57.0",
        "Cluster_LP_58.0",
        "Cluster_LP_59.0",
        "Cluster_LP_60.0",
        "Cluster_LP_63.0",
        "Cluster_LP_64.0",
        "Cluster_LP_65.0",
        "Cluster_LP_66.0",
        "Cluster_LP_67.0",
        "Cluster_LP_80.0",
        "Cluster_LP_82.0",
        "Cluster_LP_85.0",
        "Cluster_LP_89.0",
        "Cluster_LP_90.0",
        "Cluster_LP_91.0",
        "Cluster_LP_100.0",
        "Cluster_LP_101.0",
        "Cluster_LP_102.0",
        "Cluster_LP_103.0",
        "Cluster_LP_104.0",
        "Cluster_LP_105.0",
        "Cluster_LP_107.0",
        "Cluster_LP_108.0",
        "Cluster_LP_109.0",
        "Cluster_LP_110.0",
        "Cluster_LP_111.0",
        "Cluster_LP_124.0",
        "Cluster_LP_125.0",
        "Cluster_LP_129.0",
        "Cluster_LP_150.0",
        "Cluster_LP_151.0",
        "Cluster_LP_152.0",
        "Cluster_LP_154.0",
        "Cluster_LP_155.0",
        "Cluster_LP_156.0",
        "Cluster_LP_157.0",
        "Cluster_LP_158.0",
        "Cluster_LP_159.0",
        "Cluster_LP_160.0",
        "Cluster_LP_161.0",
        "Cluster_LP_163.0",
        "Cluster_LP_164.0",
        "Cluster_LP_165.0",
        "Cluster_LP_166.0",
        "Log_P_A_lag",
    ]
].astype(float)

Ind_Var_LP_lagXY = data_aveiro_LP_Robust[
    [
        "DT",
        "MCA_1",
        "MCA_2",
        "MCA_3",
        "MCA_4",
        "TAA",
        "IPI",
        "Tot_AL",
        "Cluster_LP_5.0",
        "Cluster_LP_13.0",
        "Cluster_LP_14.0",
        "Cluster_LP_18.0",
        "Cluster_LP_19.0",
        "Cluster_LP_20.0",
        "Cluster_LP_21.0",
        "Cluster_LP_22.0",
        "Cluster_LP_24.0",
        "Cluster_LP_25.0",
        "Cluster_LP_26.0",
        "Cluster_LP_27.0",
        "Cluster_LP_28.0",
        "Cluster_LP_29.0",
        "Cluster_LP_30.0",
        "Cluster_LP_31.0",
        "Cluster_LP_37.0",
        "Cluster_LP_38.0",
        "Cluster_LP_39.0",
        "Cluster_LP_40.0",
        "Cluster_LP_41.0",
        "Cluster_LP_42.0",
        "Cluster_LP_43.0",
        "Cluster_LP_45.0",
        "Cluster_LP_57.0",
        "Cluster_LP_58.0",
        "Cluster_LP_59.0",
        "Cluster_LP_60.0",
        "Cluster_LP_63.0",
        "Cluster_LP_64.0",
        "Cluster_LP_65.0",
        "Cluster_LP_66.0",
        "Cluster_LP_67.0",
        "Cluster_LP_80.0",
        "Cluster_LP_82.0",
        "Cluster_LP_85.0",
        "Cluster_LP_89.0",
        "Cluster_LP_90.0",
        "Cluster_LP_91.0",
        "Cluster_LP_100.0",
        "Cluster_LP_101.0",
        "Cluster_LP_102.0",
        "Cluster_LP_103.0",
        "Cluster_LP_104.0",
        "Cluster_LP_105.0",
        "Cluster_LP_107.0",
        "Cluster_LP_108.0",
        "Cluster_LP_109.0",
        "Cluster_LP_110.0",
        "Cluster_LP_111.0",
        "Cluster_LP_124.0",
        "Cluster_LP_125.0",
        "Cluster_LP_129.0",
        "Cluster_LP_150.0",
        "Cluster_LP_151.0",
        "Cluster_LP_152.0",
        "Cluster_LP_154.0",
        "Cluster_LP_155.0",
        "Cluster_LP_156.0",
        "Cluster_LP_157.0",
        "Cluster_LP_158.0",
        "Cluster_LP_159.0",
        "Cluster_LP_160.0",
        "Cluster_LP_161.0",
        "Cluster_LP_163.0",
        "Cluster_LP_164.0",
        "Cluster_LP_165.0",
        "Cluster_LP_166.0",
        "Log_P_A_lag",
        "MCA_1_lag",
        "MCA_2_lag",
        "MCA_3_lag",
        "MCA_4_lag",
        "Tot_AL_lag",
    ]
].astype(float)

In [ ]:
M_OLS_LP = spreg.OLS(
    Dep_Var_LP.values,  # Dependent variable
    Ind_Var_LP.values,  # Independent variable
    name_y="Log_P_A",  # Dependent variable name
    name_x=list(Ind_Var_LP.columns),  # Independent variable name
    w=w_Queen_LP,
    spat_diag=True,
    moran=True,
    name_w="w_Queen",
)

print(M_OLS_LP.summary)

In [ ]:
data_aveiro_LP_Robust["residuals_OLS"] = M_OLS_LP.u

In [ ]:
sns.displot(data_aveiro_LP_Robust["residuals_OLS"], bins=30, kde=True)
plt.title("Distributions of residuals")

In [ ]:
# standardisation of the residuals (Z-scores)
data_aveiro_LP_Robust["Z_Score_residuals_OLS"] = stats.zscore(
    data_aveiro_LP_Robust["residuals_OLS"]
)

In [ ]:
# distribution of the data against the expected normal distribution.
qqplot(data_aveiro_LP_Robust["Z_Score_residuals_OLS"], line="s")
plt.title("Normal Q-Q plot of residuals")

In [ ]:
# Create a Map - Equal Intervals
f, ax = plt.subplots(1, figsize=(8, 12))
ax = data_aveiro_LP_Robust.plot(
    column="Z_Score_residuals_OLS",  # Data to plot
    scheme="EqualInterval",  # Classification scheme
    cmap="bwr",  # Color palette
    edgecolor="k",  # Borderline color
    linewidth=0.1,  # Borderline width
    legend=True,  # Add legend
    legend_kwds={"fmt": "{:.1f}"},  # Remove decimals in legend (for legibility)
    k=10,
    ax=ax,
)

ax.set_title("residuals_OLS")
ax.set_axis_off()

In [ ]:
# Create a Map - Equal Intervals
f, ax = plt.subplots(1, figsize=(8, 12))
ax = data_aveiro_LP_Robust.plot(
    column="Z_Score_residuals_OLS",  # Data to plot
    scheme="StdMean",  # Classification scheme
    cmap="bwr",  # Color palette
    edgecolor="k",  # Borderline color
    linewidth=0.1,  # Borderline width
    legend=True,  # Add legend
    legend_kwds={
        "fmt": "{:.1f}",
        "loc": "lower right",
    },  # Remove decimals in legend (for legibility)
    ax=ax,
)

ax.set_title("residuals_OLS")
ax.set_axis_off()